**Set environment**

In [1]:
### set env
source ../config/config_duke.sh
source ../config/config_load_module_bedtools.sh
show_env

You are on Duke Server: HARDAC
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/out
CODE DIRECTORY (FD_CODE): /data/reddylab/Kuei/code
SING DIRECTORY (FD_SING): /data/reddylab/Kuei/singularity
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC
PATH OF RESULTS (FD_RES): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc
PATH OF LOG     (FD_LOG): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log



In [3]:
FD_INP=${FD_RES}/results/region/annotation_enhancer_zscore_junke
ls ${FD_INP}

description.tsv
enhancer_zscore_junke.inactive.ASTARR.bed.gz
enhancer_zscore_junke.inactive.LMPRA.bed.gz
enhancer_zscore_junke.inactive.TMPRA.bed.gz
enhancer_zscore_junke.inactive.TMPRA_OL13.bed.gz
enhancer_zscore_junke.inactive.TMPRA_OL43.bed.gz
enhancer_zscore_junke.inactive.TMPRA_OL45.bed.gz
enhancer_zscore_junke.inactive.WSTARR.bed.gz
enhancer_zscore_junke.peak.ASTARR_AB.bed.gz
enhancer_zscore_junke.peak.ASTARR_A.bed.gz
enhancer_zscore_junke.peak.ASTARR_R.bed.gz
enhancer_zscore_junke.peak.LMPRA_AB.bed.gz
enhancer_zscore_junke.peak.LMPRA_A.bed.gz
enhancer_zscore_junke.peak.LMPRA_R.bed.gz
enhancer_zscore_junke.peak.TMPRA_A.bed.gz
enhancer_zscore_junke.peak.TMPRA_R.bed.gz
enhancer_zscore_junke.peak.WSTARR_AB.bed.gz
enhancer_zscore_junke.peak.WSTARR_A.bed.gz
enhancer_zscore_junke.peak.WSTARR_R.bed.gz


In [4]:
FD_INP=${FD_RES}/results/region/annotation_enhancer_zscore_junke
FN_INP=enhancer_zscore_junke.inactive.ASTARR.bed.gz
FP_INP=${FD_INP}/${FN_INP}
fun_head ${FP_INP}

chr1	115710	115810	bin_59	0.488572939976538	-	0.0494318585308272	0.109071399098828	0.650631884838429	N	inactive	145	190	215	190	208	161	21	53	54	72
chr1	115720	115820	bin_61	0.344825933065463	-	0.20866546748985	0.338232419737616	0.5492064954387383	N	inactive	132	169	194	170	189	144	18	43	43	64
chr1	115730	115830	bin_63	0.249383142479579	+	0.441324871085628	0.5858410419082	0.4818637214319384	N	inactive	132	167	217	176	199	159	22	44	43	52
chr1	115730	115830	bin_64	0.399196858523378	-	0.109188715315645	0.204992193295846	0.5875696753413435	N	inactive	147	190	221	189	213	189	41	47	49	64
chr1	115740	115840	bin_65	0.201335693587275	+	0.489442923072408	0.629799310960666	0.44796227667644267	N	inactive	109	142	176	150	162	142	21	37	32	45
chr1	115740	115840	bin_66	0.33711287381878	-	0.199192716207137	0.3265277102148	0.5437642949062316	N	inactive	131	166	200	168	186	154	31	41	42	54
chr1	115750	115850	bin_67	0.23340999007543	-	0.452647630053691	0.596362040063265	0.47059334274827214	N	inactive	123	1

In [5]:
ASSAYS=("ASTARR" "WSTARR" "TMPRA" "LMPRA")
echo ${ASSAYS[0]}
echo ${ASSAYS[2]}

ASTARR
TMPRA


In [9]:
sbatch -p ${NODE} \
    --exclude=dl-01 \
    --cpus-per-task 8 \
    --mem 4G \
    --output ${FD_LOG}/annotation_enhancer_merge_starrmpra_inactive.%a.txt \
    --array 0-3 \
    <<'EOF'
#!/bin/bash
### set directories
source ../config/config_duke.sh

### print start message
timer_start=`date +%s`
echo "Hostname:          " $(hostname)
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### set input and output
ASSAYS=("ASTARR" "WSTARR" "TMPRA" "LMPRA")
ASSAY=${ASSAYS[${SLURM_ARRAY_TASK_ID}]}

FD_INP=${FD_RES}/results/region/annotation_enhancer_zscore_junke
FD_OUT=${FD_RES}/results/region/annotation_enhancer_zscore_junke

FN_INP=enhancer_zscore_junke.inactive.${ASSAY}.bed.gz
FN_OUT=enhancer_zscore_junke.inactive.merge.${ASSAY}.bed.gz

FP_INP=${FD_INP}/${FN_INP}
FP_OUT=${FD_OUT}/${FN_OUT}

### show I/O file
echo "Input: " ${FP_INP}
echo
echo "show first few lines of input"
fun_head ${FP_INP}
echo

### init: create output folder if not exist
mkdir -p ${FD_OUT}

### merge the regions within the bed file
bedtools merge \
    -i ${FP_INP} \
| gzip -c \
> ${FP_OUT}

### show output file
echo
echo "Output: " ${FP_OUT}
echo
echo "show first few lines of output:"
fun_head ${FP_OUT}
echo

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 30178399


In [10]:
cat ${FD_LOG}/annotation_enhancer_merge_starrmpra_inactive.0.txt

Hostname:           x1-03-3.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         08-10-23+14:36:07

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_enhancer_zscore_junke/enhancer_zscore_junke.inactive.ASTARR.bed.gz

show first few lines of input
chr1	115710	115810	bin_59	0.488572939976538	-	0.0494318585308272	0.109071399098828	0.650631884838429	N	inactive	145	190	215	190	208	161	21	53	54	72
chr1	115720	115820	bin_61	0.344825933065463	-	0.20866546748985	0.338232419737616	0.5492064954387383	N	inactive	132	169	194	170	189	144	18	43	43	64
chr1	115730	115830	bin_63	0.249383142479579	+	0.441324871085628	0.5858410419082	0.4818637214319384	N	inactive	132	167	217	176	199	159	22	44	43	52
chr1	115730	115830	bin_64	0.399196858523378	-	0.109188715315645	0.204992193295846	0.5875696753413435	N	inactive	147	190	221	189	213	189	41	47	49	64
chr1	115740	115840	bin_65	0.201335693587275	+	0.489442923072408	0.629799310960666	0.44796227667644267	N	inactive	109	142	17

In [11]:
cat ${FD_LOG}/annotation_enhancer_merge_starrmpra_inactive.1.txt

Hostname:           x1-03-4.genome.duke.edu
Slurm Array Index:  1
Time Stamp:         08-10-23+14:36:07

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_enhancer_zscore_junke/enhancer_zscore_junke.inactive.WSTARR.bed.gz

show first few lines of input
chr1	88430	88530	bin_43	0.219133859242476	+	0.735784438477576	0.910566085869422	0.2949021802449866	N	inactive	11	11	11	4
chr1	88440	88540	bin_44	0.243784462697835	+	0.705576146063813	0.898184561496427	0.34164495665210354	N	inactive	11	11	12	4
chr1	88450	88550	bin_45	0.269256185975577	+	0.676086384487412	0.885449245240551	0.3899447505306429	N	inactive	11	11	13	4
chr1	133200	133300	bin_62	0.0159003788123292	+	0.973923556971652	0.992733089896101	-0.09047163631857148	N	inactive	14	12	5	11
chr1	133200	133300	bin_63	0.184097336156335	-	0.661300909358384	0.879543475902448	0.22846549573687386	N	inactive	18	19	8	11
chr1	133210	133310	bin_64	-0.0126174104498143	+	0.979625649760522	0.994631294789604	-0.14454741785